In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
import subprocess
from subprocess import Popen, PIPE

# Chartevents further wrangling

In [ ]:
## Extract features from chart_events (still too big to upload into python)

In [2]:
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/icu'
items = pd.read_csv(path+'/d_items.csv')

In [6]:
items.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
2,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
3,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN


In [51]:
items[items['label'].str.contains('CO2')]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
30,220235,Arterial CO2 Pressure,PCO2 (Arterial),chartevents,Labs,mmHg,Numeric,NaN,NaN
31,220245,CO2 production,CO2 production,chartevents,Respiratory,ml/min,Numeric,NaN,NaN
214,223679,TCO2 (calc) Venous,TCO2 (calc) Venous,chartevents,Labs,None,Numeric,NaN,NaN
1127,225698,TCO2 (calc) Arterial,TCO2 (calc) Arterial,chartevents,Labs,None,Numeric,NaN,NaN
1341,226062,Venous CO2 Pressure,PCO2 (Venous),chartevents,Labs,mmHg,Numeric,NaN,NaN
1660,227036,PCO2_ApacheIV,PCO2_ApacheIV,chartevents,Scores - APACHE IV (2),None,Numeric,NaN,NaN
1662,227038,PHPaCO2Score_ApacheIV,PHPaCO2Score_ApacheIV,chartevents,Scores - APACHE IV (2),None,Numeric,NaN,NaN
2486,228640,EtCO2,EtCO2,chartevents,Routine Vital Signs,None,Numeric,NaN,NaN
2487,228641,EtCO2 Clinical indication,EtCO2 Clinical indication,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN
2659,228878,PeCO2,PeCO2,chartevents,Respiratory,mmHg,Numeric,NaN,NaN


In [4]:
##Name of features in table
features = ['Heart Rate', 'O2 saturation pulseoxymetry','Respiratory Rate', 'ZC Reactive Protein (CRP)'
            'Temperature Fahrenheit', 'Temperature Celsius',
            'Non Invasive Blood Pressure systolic', 'Arterial Blood Pressure systolic',
           'Arterial Blood Pressure mean', 'Arterial Blood Pressure diastolic',
           'Non Invasive Blood Pressure diastolic','Non Invasive Blood Pressure diastolic', 
           'EtCO2','Arterial Base Excess',
           'Inspired O2 Fraction', 'Arterial CO2 Pressure',
            'ART %O2 saturation (PA Line)', 'Direct Bilirubin', 'BUN', 'BUN_ApacheIV'
            'AST', 'PH', 'Chloride (serum)',
            'Glucose (serum)', 'Magnesium',
            'Alkaline Phosphate','Creatinine (whole blood)','Hematocrit (serum)',
            'Ammonia', 'Lactic Acid',
            'Hemoglobin','Total Bilirubin', 'Bilirubin_ApacheIV','Troponin-T', 'PTT', 'WBC',
            'ZFibrinogen', 'Platelet Count', 'Potassium (serum)', 'ZPotassium (serum)', 'Urine output_ApacheIV',
            'GU Irrigant/Urine Volume Out GU Irrigant/Urine Volume', 'Sodium (serum)', 'WBC', 'WBC_ApacheIV',
            'Creatinine (serum)', 'Platelet Count', 'GCS - Eye Opening', 'GCS - Verbal Response',
            'GCS - Motor Response', 'C Reactive Protein (CRP)', 'Absolute Neutrophil Count',
            'D-Dimer','LDH', 'OxygenScore_ApacheIV'
           ]

In [5]:
#extract itemids and save
features_analysis = items[items['label'].isin(features)]
features_analysis = pd.DataFrame(features_analysis['itemid'])
features_analysis.to_csv('1.0/small/icu/chartevents_features.csv', index = False)

In [85]:
#extract features from chartevents
request = 'awk -F, \'FNR==NR{a[$1]++;next}a[$6]\' chartevents_features.csv chartevents.csv > chartevents_filtered.csv'

In [6]:
##get the icu stays
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
icustays = pd.read_csv(path + 'small/icu/icustays.csv', parse_dates = ['intime', 'outtime'])
#filter the longest stay
icustays['obs_window_end'] = icustays['intime'] + timedelta(days = 7)

In [7]:
#further reduce the size of the chart events
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'

#chunk in chart events, select the latest observation in the window and save to a new file
df = pd.DataFrame()
for i, chartevents in  enumerate(pd.read_csv(path + 'small/icu/chartevents_filtered.csv', chunksize = 1000000, parse_dates = ['charttime'])):
    chartevents = chartevents.merge(icustays, on= 'hadm_id')
    df_add = chartevents[(chartevents['charttime'] >= chartevents['intime']) & (chartevents['charttime'] <= chartevents['obs_window_end'])]
    df_add = df_add.iloc[:,[0,1,3,5,6,7,8]]
    df_add = df_add.loc[df_add.groupby(['hadm_id','itemid']).charttime.idxmax()]
    df = pd.concat([df, df_add])
df.to_csv(path + 'small/icu/chartevents_filtered_use.csv', index = False)

/gpfs/commons/home/aelhussein/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Extract DICOM IDs

In [79]:
#Get the dicomids and download the specific images
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
icustays = pd.read_csv(path + 'small/icu/icustays.csv', parse_dates = ['intime', 'outtime'])
cxr = pd.read_csv(path + 'ids/mimic-cxr-2.0.0-metadata.csv', parse_dates = ['StudyDate'])

In [80]:
#filter the longest stay
icustays['obs_window_end'] = icustays['intime'] + timedelta(days = 7)
icustays['obs_window_start'] = icustays['intime'] - timedelta(days = 3)
icustays = icustays.loc[icustays.groupby('hadm_id').los.idxmax()]

In [81]:
#filter out the lateral views
cxr = cxr[(cxr['ViewPosition'] != 'LATERAL') & (cxr['ViewPosition'] != 'LL')]
cxr.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,2180-05-06,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,2180-06-26,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,2180-07-23,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN
5,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,10000032,53911762,CHEST (PORTABLE AP),AP,2906,2258,2180-07-23,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect
6,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,10000032,56699142,CHEST (PORTABLE AP),AP,3056,2544,2180-08-05,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN


In [82]:
#filter cxr for our subjects
cxr_filtered = cxr[cxr['subject_id'][(cxr['ViewPosition'] != 'LATERAL') & (cxr['ViewPosition'] != 'LL')].isin(icustays['subject_id'])]

In [84]:
# merge with icustays
cxr_filtered = cxr_filtered.merge(icustays, on = 'subject_id')

In [86]:
##
cxr_filtered = cxr_filtered[(cxr_filtered['obs_window_end'] >= cxr_filtered['StudyDate']) & 
                             (cxr_filtered['obs_window_start'] <= cxr_filtered['StudyDate'])]
cxr_use = cxr_filtered.loc[cxr_filtered.groupby('subject_id').StudyDate.idxmax()]

In [88]:
cxr_use.to_csv(path + 'cxr_metadata_use2.csv', index = False)

In [89]:
#create the naming logic needed to download
cxr_use['folder'] = 'p' + cxr_use['subject_id'].astype(str).str[:2]
cxr_use['sub_folder'] = 'p' + cxr_use['subject_id'].astype(str)
cxr_use['study_folder'] = 's' + cxr_use['study_id'].astype(str)

In [90]:
#create file request
request = 'wget -r -N -c -np --user ahmed-elhussein --ask-password '
files = ''
for i, row in cxr_use.iterrows():
    file = 'https://physionet.org/files/mimic-cxr-jpg/2.0.0/files/{}/{}/{}/{}.jpg '.format(
        row['folder'], row['sub_folder'], row['study_folder'], row['dicom_id'])
    files = files + file
    
request = request+files

# demographic features

In [8]:
#extract demographic features
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
patients = pd.read_csv(path + 'core/patients.csv', parse_dates = ['anchor_year'])
admissions = pd.read_csv(path + 'core/admissions.csv', parse_dates = ['admittime'])

In [23]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [9]:
#merge with admissions and patients to get all the demographic data needed
df = icustays.merge(patients, on = 'subject_id')
df = df.merge(admissions, on = ['subject_id','hadm_id'])

In [ ]:
df.head()

In [10]:
#create new demographic variables
df['age'] = df['anchor_age']+(df['intime'] - df['anchor_year'])/timedelta(days = 365)
df['hours_since_admit'] = (df['intime'] - df['admittime']).dt.total_seconds() / 3600

In [11]:
demographic = df[['subject_id', 'hadm_id', 'age', 'first_careunit',
   'gender','admittime', 'dischtime','deathtime', 'hours_since_admit']]

demographic = demographic.loc[demographic.groupby('hadm_id').hours_since_admit.idxmin()]

## create final dataset

In [12]:
#create the final dataset
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
df = pd.read_csv(path + 'small/icu/chartevents_filtered_use.csv', parse_dates = ['charttime'])

In [20]:
df.head()

,subject_id_x,hadm_id,charttime,itemid,value,valuenum,valueuom
0,10581271,20037205,2122-03-08 21:00:00,220045,82.0,82.0,bpm
1,10581271,20037205,2122-03-08 21:02:00,220179,85.0,85.0,mmHg
2,10581271,20037205,2122-03-08 21:02:00,220180,72.0,72.0,mmHg
3,10581271,20037205,2122-03-08 03:43:00,220228,11.0,11.0,g/dl
4,10581271,20037205,2122-03-07 02:16:00,220235,38.0,38.0,mmHg


In [13]:
##values that are not numeric
df['itemid'][pd.to_numeric(df['value'], errors='coerce').isnull()].unique()

array([220739, 223900, 223901])

In [14]:
#create a pivot table where each row is an admission and each column is the feature
df = df.iloc[df.groupby(['hadm_id', 'itemid']).charttime.idxmax()]
df_pivot = df.pivot_table(index = 'hadm_id', columns = 'itemid', values = 'valuenum', aggfunc = 'mean')

In [31]:
df_pivot

itemid,220045,220050,220051,220052,220179,220180,220228,220235,220277,220545,...,225612,225624,225651,225668,225690,227429,227442,227457,227466,228640
hadm_id,,,,,,,,,,,,,,,,,,,,,
20001305,69.0,NaN,NaN,NaN,47.0,36.0,10.1,NaN,94.0,34.5,...,281.0,59.0,NaN,2.0,0.5,0.44,4.3,170.0,150.0,28.0
20005119,76.0,146.0,82.0,110.0,109.0,68.0,8.5,38.0,98.0,25.5,...,NaN,19.0,NaN,0.4,NaN,NaN,3.3,128.0,26.7,NaN
20013496,78.0,142.0,137.0,138.0,106.0,58.0,7.5,46.0,100.0,23.4,...,NaN,14.0,NaN,1.9,NaN,NaN,3.3,173.0,20.4,NaN
20013839,108.0,NaN,NaN,NaN,111.0,62.0,6.6,46.0,98.0,22.5,...,NaN,12.0,NaN,1.1,NaN,NaN,3.7,343.0,28.7,NaN
20015802,115.0,NaN,NaN,NaN,79.0,51.0,9.9,60.0,96.0,31.4,...,NaN,30.0,NaN,0.7,NaN,0.01,4.0,191.0,24.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993312,99.0,NaN,NaN,NaN,126.0,68.0,8.6,49.0,94.0,26.7,...,75.0,80.0,NaN,0.9,0.4,NaN,4.6,202.0,26.4,NaN
29994310,79.0,NaN,NaN,NaN,119.0,61.0,11.2,NaN,94.0,32.5,...,139.0,13.0,NaN,NaN,2.8,0.02,3.4,200.0,28.7,NaN
29995768,82.0,102.0,58.0,76.0,NaN,NaN,10.9,43.0,97.0,33.0,...,66.0,10.0,NaN,0.9,0.2,NaN,3.4,171.0,35.4,NaN


In [15]:
##filter out features with >20% missing
cols_to_use = df_pivot.columns[df_pivot.isna().sum(axis = 0) / df_pivot.shape[0] < 0.3]
cols_dropped = df_pivot.columns[df_pivot.isna().sum(axis = 0) / df_pivot.shape[0] >= 0.3]
df_pivot = df_pivot.loc[:, cols_to_use]

In [16]:
#features to be used
items[items['itemid'].isin(cols_to_use)]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
22,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
23,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
29,220228,Hemoglobin,Hemoglobin,chartevents,Labs,g/dl,Numeric,NaN,NaN
30,220235,Arterial CO2 Pressure,PCO2 (Arterial),chartevents,Labs,mmHg,Numeric,NaN,NaN
33,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
40,220545,Hematocrit (serum),Hematocrit (serum),chartevents,Labs,None,Numeric,NaN,NaN
41,220546,WBC,WBC,chartevents,Labs,None,Numeric,NaN,NaN
49,220602,Chloride (serum),Chloride (serum),chartevents,Labs,None,Numeric,NaN,NaN
53,220615,Creatinine (serum),Creatinine (serum),chartevents,Labs,None,Numeric,NaN,NaN


In [17]:
#features dropped
items[items['itemid'].isin(cols_dropped)]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
5,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
6,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
7,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
46,220580,Ammonia,Ammonia,chartevents,Labs,None,Numeric,NaN,NaN
56,220632,LDH,LDH,chartevents,Labs,None,Numeric,NaN,NaN
221,223762,Temperature Celsius,Temperature C,chartevents,Routine Vital Signs,°C,Numeric,NaN,NaN
1077,225612,Alkaline Phosphate,Alkaline Phosphate,chartevents,Labs,None,Numeric,NaN,NaN
1087,225636,D-Dimer,D-Dimer,chartevents,Labs,None,Numeric,NaN,NaN
1097,225651,Direct Bilirubin,Direct Bilirubin,chartevents,Labs,None,Numeric,NaN,NaN
1122,225690,Total Bilirubin,Total Bilirubin,chartevents,Labs,None,Numeric,NaN,NaN


In [18]:
##add in the demographic features
df_features = df_pivot.merge(demographic[['hadm_id', 'age', 'first_careunit',
   'gender', 'hours_since_admit']], left_index = True, right_on = 'hadm_id')

In [19]:
df_features.set_index('hadm_id', inplace = True)

In [20]:
#replace location with either medical or surgical unit
locations = {'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU',
       'Trauma SICU (TSICU)':'SICU', 'Coronary Care Unit (CCU)':'MICU',
       'Medical Intensive Care Unit (MICU)':'MICU',
       'Cardiac Vascular Intensive Care Unit (CVICU)':'MICU',
       'Surgical Intensive Care Unit (SICU)':'SICU',
       'Neuro Surgical Intensive Care Unit (Neuro SICU)':'SICU',
       'Neuro Stepdown':'SICU'}

In [21]:
df_features['first_careunit'] = df_features['first_careunit'].replace(locations)

In [22]:
#create one-hot encoding
df_features = pd.get_dummies(df_features, columns = ['first_careunit', 'gender'], drop_first = True)

In [57]:
df_features

,220045,220179,220180,220228,220235,220277,220545,220546,220602,220615,...,224828,225624,225668,227442,227457,227466,age,hours_since_admit,first_careunit_SICU,gender_M
hadm_id,,,,,,,,,,,,,,,,,,,,,
20001305,69.0,47.0,36.0,10.1,NaN,94.0,34.5,12.8,109.0,1.2,...,NaN,59.0,2.0,4.3,170.0,150.0,84.233218,0.019167,0,0
20005119,76.0,109.0,68.0,8.5,38.0,98.0,25.5,16.3,127.0,0.4,...,-1.0,19.0,0.4,3.3,128.0,26.7,56.857217,0.800000,1,0
20013496,78.0,106.0,58.0,7.5,46.0,100.0,23.4,10.3,104.0,0.5,...,3.0,14.0,1.9,3.3,173.0,20.4,21.841891,2.416667,1,0
20013839,108.0,111.0,62.0,6.6,46.0,98.0,22.5,21.5,104.0,0.6,...,5.0,12.0,1.1,3.7,343.0,28.7,65.513185,1.233333,1,0
20015802,115.0,79.0,51.0,9.9,60.0,96.0,31.4,7.4,96.0,1.0,...,11.0,30.0,0.7,4.0,191.0,24.2,71.065313,0.026944,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993312,99.0,126.0,68.0,8.6,49.0,94.0,26.7,8.4,102.0,6.6,...,0.0,80.0,0.9,4.6,202.0,26.4,72.799365,0.801667,0,1
29994310,79.0,119.0,61.0,11.2,NaN,94.0,32.5,9.8,103.0,0.8,...,NaN,13.0,NaN,3.4,200.0,28.7,79.432512,42.991944,0,1
29995768,82.0,119.0,73.0,10.9,43.0,97.0,33.0,14.7,131.0,0.8,...,0.0,10.0,0.9,3.4,171.0,35.4,58.372112,0.933333,1,0


In [24]:
## use random forest imputation for missing values
imputer = MissForest()
df_imputed = imputer.fit_transform(df_features)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [25]:
df_imputed = pd.DataFrame(df_imputed, columns = df_features.columns, index = df_features.index)

In [26]:
gcs = [223900,223901,220739]
df_imputed['gcs'] = df_imputed.loc[:,gcs[0]] + df_imputed.loc[:,gcs[1]] + df_imputed.loc[:,gcs[2]]
df_imputed_cleaned = df_imputed.drop(columns = gcs)

In [27]:
##bucket gcs
bins = [0,5,8,12,15]
labels = [0,1,2,3]
df_imputed_cleaned['gcs_cat'] = pd.cut(df_imputed_cleaned['gcs'], bins, labels = labels)
df_imputed_cleaned.drop('gcs', axis = 1, inplace = True)
df_imputed_cleaned = pd.get_dummies(df_imputed_cleaned, columns = ['gcs_cat'], drop_first = True)
df_imputed_cleaned = df_imputed_cleaned.rename(columns = {'gcs_cat_1': 'gcs_5-8', 'gcs_cat_2': 'gcs_9-12','gcs_cat_3': 'gcs_13-15'})

In [58]:
df_imputed_cleaned.head()

,220045,220179,220180,220228,220235,220277,220545,220546,220602,220615,...,227442,227457,227466,age,hours_since_admit,first_careunit_SICU,gender_M,gcs_5-8,gcs_9-12,gcs_13-15
hadm_id,,,,,,,,,,,,,,,,,,,,,
20001305,69.0,47.0,36.0,10.1,42.82,94.0,34.5,12.8,109.0,1.2,...,4.3,170.0,150.0,84.233218,0.019167,0.0,0.0,1,0,0
20005119,76.0,109.0,68.0,8.5,38.00,98.0,25.5,16.3,127.0,0.4,...,3.3,128.0,26.7,56.857217,0.800000,1.0,0.0,0,0,0
20013496,78.0,106.0,58.0,7.5,46.00,100.0,23.4,10.3,104.0,0.5,...,3.3,173.0,20.4,21.841891,2.416667,1.0,0.0,0,1,0
20013839,108.0,111.0,62.0,6.6,46.00,98.0,22.5,21.5,104.0,0.6,...,3.7,343.0,28.7,65.513185,1.233333,1.0,0.0,0,0,1
20015802,115.0,79.0,51.0,9.9,60.00,96.0,31.4,7.4,96.0,1.0,...,4.0,191.0,24.2,71.065313,0.026944,0.0,1.0,0,1,0


In [28]:
df_imputed_cleaned.to_csv(path + 'small/dataset2.csv')